## IBM Capstone Project 2 - Toronto

### Author: Sergey Kosenko

## 1. Wiki page parsing

1. Start by creating a new Notebook for this assignment.
2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

In [11]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#!conda install -c conda-forge lxml -- yes # if lxml is not found

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

!pip install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [28]:
#https://pbpython.com/pandas-html-table.html
wiki_page = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
print(f'Total tables exported from the wiki page: {len(wiki_page)}')

Total tables exported from the wiki page: 3


In [83]:
## 3
#1. Create a dataframe with three columns: PostalCode, Borough, and Neighborhood
df_toronto = wiki_page[0]

#2. Ignore cells with a borough that is Not assigned.
df_toronto = df_toronto[(df_toronto.Borough != 'Not assigned')]

#3. If a Neighbourhood has the 'Not assigned' value has a borough, then the neighborhood should be replaced with the same value as the borough.
print("A number of 'Not Assigned' neighborhoods:", df_toronto[(df_toronto.Neighbourhood == 'Not assigned')].shape[0], '. No further changes are required here.')

# check resulting dataframe
df_toronto.head()

A number of 'Not Assigned' neighborhoods: 0 . No further changes are required here.


,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [73]:
#print the number of rows of your dataframe
df_toronto.shape

(103, 3)

### 2. Adding geocoding data

In [58]:
!pip install geocoder
import geocoder # import geocoder

     |████████████████████████████████| 98 kB 7.7 MB/s  eta 0:00:01


As geocoder works unstable (I could not get locations even for one postal) - I will you the prepared data from CSV file by the provided link: https://cocl.us/Geospatial_data

In [74]:
geo_data = pd.read_csv('https://cocl.us/Geospatial_data')

In [84]:
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [85]:
#merge df_toronto and geo_data dataframes into one dataframe
df_toronto = pd.merge(df_toronto, geo_data, on='Postal Code', how='outer')

In [99]:
df_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
